## Periods in Rubin compared to PanSTARRS

In this notebook we will compare periods for objects in Rubin with those we get from PanSTARRS.

In [ ]:
import astropy.units as u
import lsdb
import matplotlib.pyplot as plt
import numpy as np
import tempfile

from astropy.timeseries import LombScargle
from dask.distributed import Client
from nested_pandas import NestedDtype
from pathlib import Path

In [ ]:
def cast_nested(df, columns):
    return df.assign(
        **{
            col: df[col].astype(NestedDtype.from_pandas_arrow_dtype(df.dtypes[col]))
            for col in columns
        },
    )

def create_mag_errors(flux, fluxerr):
    """Move flux into magnitudes and calculate the error on the magnitude"""
    mag = u.Jy.to(u.ABmag, flux)
    upper_mag = u.Jy.to(u.ABmag, flux + fluxerr)
    lower_mag = u.Jy.to(u.ABmag, flux - fluxerr)
    magErr = -(upper_mag - lower_mag) / 2
    return mag, magErr

In [ ]:
tmp_path = tempfile.TemporaryDirectory()
client = Client(n_workers=4, threads_per_worker=1, local_directory=tmp_path.name)

### Load Rubin and PS1

In [ ]:
drp_release = "w_2025_10"
base_dir = Path("/sdf/data/rubin/shared/lsdb_commissioning/hats")
hats_dir = base_dir / drp_release

In [ ]:
rubin_periods = lsdb.read_hats("rubin_periods", columns=["index","ra","dec","objectId","forcedSource","period","period_0"])
# We use the `cast_nested` utility method to cast columns into the NestedFrame type
rubin_periods = rubin_periods.map_partitions(cast_nested, columns=["forcedSource"])

We will read `object_lc_x_ps1` which we had already computed for the result of the crossmatch of Rubin with PS1.

In [ ]:
object_lc_x_ps1 = lsdb.read_hats(hats_dir / "object_lc_x_ps1", columns=["objectId", "objID_ps1"])

This catalog does not contain the PS1 detections so let's add them as a nested column:

In [ ]:
ps1_detections = lsdb.read_hats(
    's3://stpubdata/panstarrs/ps1/public/hats/detection',
    margin_cache='s3://stpubdata/panstarrs/ps1/public/hats/detection_10arcs',
    columns=["objID", "filterID", "obsTime", "psfFlux", "psfFluxErr"]
)

Let's join the Rubin objects with PS1:

In [ ]:
object_lc_x_ps1 = object_lc_x_ps1.join_nested(ps1_detections, left_on="objID_ps1", right_on="objID", nested_column_name="ps1_lc")
object_lc_x_ps1

We can now crossmatch our catalog of Rubin periods with `object_lc_x_ps1` to get the aggregated data that we need:

In [ ]:
variables_cat = rubin_periods.join(object_lc_x_ps1, left_on="objectId", right_on="objectId", suffixes=("_rubin",""))
variables_cat

### Calculate periods for PS1 and compare with Rubin's

In [ ]:
def compare_periodograms(index_rubin, true_period, ps1_time, ps1_flux, rubin_time, rubin_flux):
    # define a freq grid to search around the true period/frequency
    true_freq = 1/true_period
    frequency = np.linspace(0.9*true_freq, 1.1*true_freq, 100)
    # Period for PS1
    ps1_power = LombScargle(ps1_time, ps1_flux).power(frequency)
    ps1_maxpower = np.max(ps1_power)
    ps1_period = 1/frequency[np.argmax(ps1_power)]
    # Period for Rubin
    rubin_power = LombScargle(rubin_time, rubin_flux).power(frequency)
    rubin_maxpower = np.max(rubin_power)
    rubin_period = 1/frequency[np.argmax(rubin_power)]
    return {
        "index_rubin": index_rubin, 
        "true_period": true_period,
        "ps1_maxpower": ps1_maxpower,
        "ps1_period": ps1_period,
        "rubin_maxpower": rubin_maxpower,
        "rubin_period": rubin_period
    }

In [ ]:
# Use the "r" band only to get the periods
r_band = variables_cat.query("ps1_lc.filterID == 1").compute()

In [ ]:
features = r_band.reduce(
    compare_periodograms,
    "index_rubin",
    "period_rubin",
    "ps1_lc.obsTime",
    "ps1_lc.psfFlux",
    "forcedSource_rubin.midpointMjdTai",
    "forcedSource_rubin.psfFlux",
).set_index("index_rubin")

features

### Plot original and phased light curves

Now let's plot light curves for each object on both surveys:

In [ ]:
# Rubin has "ugrizy" filters
COLORS = {
    "u": "#56b4e9",
    "g": "#009e73",
    "r": "#f0e442",
    "i": "#cc79a7",
    "z": "#d55e00",
    "y": "#0072b2",
}

# PanSTARRS has "grizy" filters [0..4]
PS1_COLORS = {i: color for i, color in enumerate("grizy")}

In [ ]:
def plot_ps1(ax, lc, period):
    mjd_col = "obsTime"
    flux_col = "psfFlux"
    fluxerr_col = "psfFluxErr"
    # Plot original light curve
    ax[0].set_title("PS1 - Original")
    mags = _plot_ps1_lc(ax[0], lc, flux_col, fluxerr_col, x_name=mjd_col, x_label="MJD")
    # Plot phased light curve
    ax[1].set_title(f"PS1 - Phase folded ({period})")
    lc = lc.assign(phase=(lc[mjd_col] - lc[mjd_col].loc[lc[flux_col].idxmax()]) % period / period)
    _plot_ps1_lc(ax[1], lc, flux_col, fluxerr_col, x_name="phase", x_label="Phase")
    scale_y_axis(ax, mags)
    scale_x_axis(ax, lc[mjd_col], lc["phase"], delta=200)

def plot_rubin(ax, lc, period):
    mjd_col = "midpointMjdTai"
    mag_col ="psfMag"
    magerr_col = "psfMagErr"
    # Plot original light curve
    ax[0].set_title("Rubin - Original")
    mags = _plot_rubin_lc(ax[0], lc, mag_col, magerr_col, x_name=mjd_col, x_label="MJD")
    # Plot phased light curve
    ax[1].set_title(f"Rubin - Phase folded ({period})")
    lc = lc.assign(phase=(lc[mjd_col] - lc[mjd_col].loc[lc[mag_col].idxmax()]) % period / period)
    _plot_rubin_lc(ax[1], lc, mag_col, magerr_col, x_name="phase", x_label="Phase")
    scale_y_axis(ax, mags)
    scale_x_axis(ax, lc[mjd_col], lc["phase"], delta=5)

def _plot_ps1_lc(ax, lc, flux_col, fluxerr_col, x_name, x_label):
    mag_values = []
    for filterid, band in PS1_COLORS.items():
        band_lc = lc.query(f"filterID == {filterid}")
        mag, magerr = create_mag_errors(band_lc[flux_col], band_lc[fluxerr_col])
        ax.errorbar(
            band_lc[x_name],
            mag,
            magerr,
            fmt="o",
            label=band,
            color=COLORS[band],
            alpha=1,
            markersize=5,
            capsize=3,
            elinewidth=1,
        )
        mag_values.extend(mag.dropna().values)
    ax.set_xlabel(x_label)
    ax.set_ylabel("Magnitude (AB)")
    ax.invert_yaxis()
    ax.legend(loc="lower right", fontsize=12)
    return mag_values

def _plot_rubin_lc(ax, lc, mag_col, magerr_col, x_name, x_label):
    mag_values = []
    for band, color in COLORS.items():
        band_lc = lc.query(f"band == '{band}'")
        mag, magerr = band_lc[mag_col], band_lc[magerr_col]
        ax.errorbar(
            band_lc[x_name],
            mag,
            magerr,
            fmt="o",
            label=band,
            color=color,
            alpha=1,
            markersize=5,
            capsize=3,
            elinewidth=1,
        )
        mag_values.extend(mag.dropna().values)
    ax.set_xlabel(x_label)
    ax.set_ylabel("Magnitude (AB)")
    ax.invert_yaxis()
    ax.legend(loc="lower right", fontsize=12)
    return mag_values
    
def scale_x_axis(ax, mjd_values, phase_values, delta):
    # Apply limits to the mjd axis
    xmin, xmax = np.nanmin(mjd_values), np.nanmax(mjd_values)
    ax[0].set_xlim(xmin - delta, xmax + delta)
    # Apply limits to the phase axis
    xmin, xmax = np.nanmin(phase_values), np.nanmax(phase_values)
    ax[1].set_xlim(xmin, xmax)

def scale_y_axis(ax, all_mags):
    # Apply limits to all columns in the row
    ymin, ymax = np.nanmin(all_mags), np.nanmax(all_mags)
    lower_bound = np.quantile(all_mags, 0.1)
    upper_bound = np.quantile(all_mags, 0.9)
    for i in range(2):
        # Keep magnitude inverted
        ax[i].set_ylim(lower_bound - 0.2, upper_bound + 0.2)
    #print("Min | Max:", ymin, ymax)
    #print("10% | 90%:", lower_bound, upper_bound)

In [ ]:
# Compute the result for the 10 variable objects
data = variables_cat.compute().set_index("index_rubin")

In [ ]:
for index_rubin, row in data.iterrows():
    object_features = features.loc[index_rubin]

    fig, ax = plt.subplots(2, 2, figsize=(12, 8), sharey="col")
    ra, dec = row.ra_rubin, row.dec_rubin

    title = f"{drp_release} | RA={ra:.5f}, Dec={dec:.5f} (T={object_features["true_period"]})"
    fig.suptitle(title, fontsize=16)

    ps1_period = round(object_features["ps1_period"], 5)
    rubin_period = round(object_features["rubin_period"], 5)
    plot_ps1(ax[0], row["ps1_lc"], ps1_period)
    plot_rubin(ax[1], row["forcedSource_rubin"], rubin_period)
    
    plt.tight_layout()
    plt.show()

In [ ]:
client.close()
tmp_path.cleanup()